# 処理パタン

処理パタンは決まっていて、それを呼び出すだけ。

## GPU設定

In [23]:
import torch

In [ ]:
def get_device():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    print(f'device count: {torch.cuda.device_count()}')
    print(f'device name: {torch.cuda.get_device_name()}')
    print(f'device capability: {torch.cuda.get_device_capability()}')

## データ前処理

In [19]:
import torchvision.transforms as transforms

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### １階テンソル化

In [21]:
transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.Lambda(lambda x: x.view(-1)),
])

### 正規化のみ

In [22]:
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

## モデル定義

In [17]:
from torch import nn

In [18]:
class Net(nn.Module):
    '''
    隠れ層が１層のNN
    '''
    def __init__(self, n_input, n_hidden, n_output):
        super(Net, self).__init__()
        self.l1 = nn.Linear(n_input, n_hidden)
        self.relu = nn.ReLU(inplace=True)
        self.l2 = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        return x

## モデル表示

In [15]:
def print_model_parameters(net):
    for parameter in net.named_parameters():
        print(parameter)

## 損失計算

In [14]:
from torchviz import make_dot

In [1]:
def eval_loss_for_computational_graph(loader, device, net, criterion):
    # データローダから最初の１セット取得（計算グラフ作成のため）
    for images, labels in loader:
        break
    # デバイス（GPU/CPU割当）
    inputs = images.to(device)
    labels = labels.to(device)
    # 予測計算
    outputs = net(inputs)
    # 損失計算
    loss = criterion(outputs, labels)
    # 計算グラフの表示
    make_dot(loss, params=dict(net.named_parameters()))
    
    return loss

## 学習・追加学習用関数

In [8]:
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
history = np.zeros((0, 5))

In [9]:
def fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history):
    base_epochs = len(history)

    for epoch in range(base_epochs, num_epochs + base_epochs):
        # 1エポックあたりの正解数（精度計算用）
        n_train_acc, n_val_acc = 0, 0
        # 1エポックあたりの累積損失（平均化前）
        train_loss, val_loss = 0, 0
        # 1エポックあたりのデータ数
        n_train, n_test = 0, 0

        ##### 訓練 #####
        net.train() # ドロップアウト関数nn.DropoutやBN関数nn.BatchNorm2dに訓練中であることを伝える関数

        for inputs, labels in tqdm(train_loader):
            train_batch_size = len(labels)
            n_train += train_batch_size

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            predicted = torch.max(outputs, 1)[1]
            train_acc += (predicted == labels).sum().item()
            train_loss += loss.item() * train_batch_size

        train_acc /= n_train
        train_loss /= n_train

        ##### 予測 #####
        net.eval() # ドロップアウト関数nn.DropoutやBN関数nn.BatchNorm2dに予測中であることを伝える関数

        for inputs_test, labels_test in test_loader:
            test_batch_size = len(labels_test)
            n_test += test_batch_size

            inputs_test = inputs_test.to(device)
            labels_test = labels_test.to(device)

            outputs_test = net(inputs_test)
            loss_test = criterion(outputs_test, labels_test)

            predict_test = torch.max(outputs_test, 1)[1]
            val_acc += (predict_test == labels_test).sum().item()
            val_loss += loss_test.item() * test_batch_size

        val_acc /= n_test
        val_loss /= n_test

        item = np.array([epoch, train_loss, train_acc, val_loss, val_acc])
        history = np.vstack((history, item))
        if epoch % (num_epochs / 100) == 0:
            print (f'Epoch [{epoch}/{num_epochs}], loss: {train_loss:.5f} acc: {train_acc:.5f} val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}')

    return history


## 学習ログ

In [6]:
def evaluate_history(history):
    pass

In [10]:
import matplotlib.pyplot as plt

In [12]:
def plot_loss_history(history):
    plt.plot(history[:, 0], history[:, 1], label='train loss')
    plt.plot(history[:, 0], history[:, 3], label='val loss')
    plt.legend()
    plt.title('Learning Curve wrt Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

In [13]:
def plot_accuracy_history(history):
    plt.plot(history[:, 0], history[:, 2], label='train acc')
    plt.plot(history[:, 0], history[:, 4], label='val acc')
    plt.legend()
    plt.title('Learning Curve wrt Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show()

## 予測結果表示（画像分析結果）

## 乱数を固定する（結果の再現性を担保するため）

In [7]:
def torch_seed(seed=123):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # GPUで結果がdeterministicになるように
    torch.use_deterministic_algorithms = True